In [ ]:
!pip install snownlp

# 自然语言情感分析

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import jieba
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedShuffleSplit, cross_validate
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_absolute_error, r2_score, make_scorer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from itertools import combinations
from sklearn.base import clone
import itertools
from itertools import chain
from joblib import Memory
from statsmodels.stats.outliers_influence import variance_inflation_factor
from category_encoders import TargetEncoder
from tqdm import tqdm
from snownlp import SnowNLP
from scipy.sparse import save_npz, load_npz

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']  # 设置中文字体

In [ ]:
# 导入训练集与测试集，使用深度拷贝保留原数据
rent_train_pd = pd.read_csv("/kaggle/input/python-ai-midterm/ruc_Class25Q2_train_rent.csv")
rent_train_raw = rent_train_pd.copy()

price_train_pd = pd.read_csv("/kaggle/input/python-ai-midterm/ruc_Class25Q2_train_price.csv")
price_train_raw = price_train_pd.copy()

rent_test_pd = pd.read_csv("/kaggle/input/python-ai-midterm/ruc_Class25Q2_test_rent.csv")
rent_test = rent_test_pd.copy()

price_test_pd = pd.read_csv("/kaggle/input/python-ai-midterm/ruc_Class25Q2_test_price.csv")
price_test = price_test_pd.copy()

In [ ]:
col_emo_price = ["客户反馈"]
col_emo_rent = ["客户反馈"]

def sentiment_score(text):
    try:
        if pd.isna(text) or str(text).strip() == "":
            return np.nan
        s = SnowNLP(str(text))
        return s.sentiments
    except Exception as e:
        print("Error:", e, " | Text:", text)
        return np.nan

for col in col_emo_rent:
    rent_train_raw[f"{col}_sentiment"] = rent_train_raw[col].apply(sentiment_score)
    rent_test[f"{col}_sentiment"] = rent_test[col].apply(sentiment_score)

for col in col_emo_price:
    price_train_raw[f"{col}_sentiment"] = price_train_raw[col].apply(sentiment_score)
    price_test[f"{col}_sentiment"] = price_test[col].apply(sentiment_score)

In [ ]:
# 提取 rent 数据的情感列
rent_sent_cols = [c for c in rent_train.columns if "sentiment" in c]
price_sent_cols = [c for c in price_train.columns if "sentiment" in c]

rent_sent_df = rent_train[rent_sent_cols].copy()
price_sent_df = price_train[price_sent_cols].copy()

# 存成独立文件（推荐 parquet 或 csv）
rent_sent_df.to_csv("rent_sentiment.csv", index=False)
price_sent_df.to_csv("price_sentiment.csv", index=False)